In [ ]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()

tr_x = tr_x * 1/255
te_x = te_x * 1/255

batch_size = 128

tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

In [ ]:
import resnet

model = resnet.wide_resnet50_2(input_shape = (32, 32, 3), include_top = False, weights = "imagenet")

flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
drop_out = tf.keras.layers.Dropout(0.5)(flatten)
dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
model = tf.keras.Model(model.input, prediction)

In [ ]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(1e-4)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, optimizer = opt, metrics = metric)

In [ ]:
model.fit(tr_data, validation_data = te_data, epochs = 10)

Epoch 1/10
391/391 [==============================] - 139s 325ms/step - loss: 1.2943 - sparse_categorical_accuracy: 0.5531 - val_loss: 0.7941 - val_sparse_categorical_accuracy: 0.7319
Epoch 2/10
391/391 [==============================] - 123s 315ms/step - loss: 0.6288 - sparse_categorical_accuracy: 0.7868 - val_loss: 0.6402 - val_sparse_categorical_accuracy: 0.7902
Epoch 3/10
391/391 [==============================] - 123s 315ms/step - loss: 0.3778 - sparse_categorical_accuracy: 0.8714 - val_loss: 0.6592 - val_sparse_categorical_accuracy: 0.8043
Epoch 4/10
391/391 [==============================] - 123s 315ms/step - loss: 0.2549 - sparse_categorical_accuracy: 0.9138 - val_loss: 0.6968 - val_sparse_categorical_accuracy: 0.8074
Epoch 5/10
391/391 [==============================] - 123s 314ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9365 - val_loss: 0.6669 - val_sparse_categorical_accuracy: 0.8156
Epoch 6/10
391/391 [==============================] - 123s 313ms/step - loss: 0.

In [ ]:
with open("model.json", mode = "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [ ]:
with open("model.json", mode = "r") as file:
    model = tf.keras.models.model_from_json(file.read())
model.load_weights("model.h5")

In [ ]:
loss = tf.keras.losses.sparse_categorical_crossentropy
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, metrics = metric)
model.evaluate(te_data)

79/79 [==============================] - 8s 73ms/step - loss: 0.7262 - sparse_categorical_accuracy: 0.8353


[0.726203441619873, 0.8353000283241272]